In [ ]:
import spacy

from scattertext import SampleCorpora, produce_scattertext_explorer

from scattertext import produce_scattertext_html
from scattertext.CorpusFromPandas import CorpusFromPandas

nlp = spacy.load('en_core_web_trf')

In [ ]:
import nltk
from nltk.book import text9
text = text9


In [ ]:
""" convention_df = SampleCorpora.ConventionData2012.get_data()
corpus = CorpusFromPandas(convention_df,
                          category_col='party',
                          text_col='text',
                          nlp=nlp).build()

html = produce_scattertext_html(corpus,
                                    category='democrat',
                                    category_name='Democratic',
                                    not_category_name='Republican',
                                    minimum_term_frequency=5,
                                    #pmi_filter_thresold=4,
                                    width_in_pixels=1000,height_in_pixels=1200) """

In [ ]:
# convert Text into a list of sentences
raw_text = text.generate(length = 5000, random_seed=22230).split()

In [ ]:
doc = nlp(' '.join(raw_text))

In [ ]:
import pandas as pd

# create an empty dataframe with required columns
df = pd.DataFrame(columns=['TokenText', 'TokenPOS', 'TokenDep'])




In [ ]:
# loop through the tokens and append values to the dataframe
for token in doc:
    token_text = token.text
    token_pos = token.pos_
    token_dep = token.dep_
    #df = df.append({'TokenText': token_text, 'TokenPOS': token_pos, 'TokenDep': token_dep}, ignore_index=True)
    if token_pos == 'NOUN' or token_pos == 'VERB' :
        new_row = pd.DataFrame([[token_text, token_pos, token_dep]], columns=['TokenText', 'TokenPOS', 'TokenDep'])
        df = pd.concat([df, new_row], ignore_index=True)

# print the dataframe
print(df)


In [ ]:
corpus = CorpusFromPandas(df,
                          category_col='TokenPOS',
                          text_col='TokenText',
                          nlp=nlp).build()

In [ ]:
html = produce_scattertext_html(corpus,
                                    category='NOUN',
                                    category_name='Nouns',
                                    not_category_name='Verbs',
                                    minimum_term_frequency=5,
                                    #pmi_filter_thresold=4,
                                    width_in_pixels=1000,height_in_pixels=1200)
open('./simple.html', 'wb').write(html.encode('utf-8'))

In [ ]:
corpus.get_categories()

In [22]:
import scattertext as st
dispersion = st.Dispersion(corpus)

dispersion_df = dispersion.get_df()
dispersion_df.head(3)


c:\Source\Repos\scattertext\scattertext\dispersion\Dispersion.py:55: RuntimeWarning: divide by zero encountered in divide
  self.p = X.multiply(csc_matrix(1. / X.sum(axis=1)))
c:\Source\Repos\scattertext\scattertext\dispersion\Dispersion.py:137: RuntimeWarning: divide by zero encountered in divide
  vfs = vf.multiply(1. / self.s)


,Frequency,Range,SD,VC,Juilland's D,Rosengren's S,DP,DP norm,KL-divergence
gates,1,1,0.0274,36.469165,2.220446e-16,0.000754,0.999246,0.999246,10.372865
brass,1,1,0.0274,36.469165,2.220446e-16,0.000754,0.999246,0.999246,10.372865
band,1,1,0.0274,36.469165,2.220446e-16,0.000754,0.999246,0.999246,10.372865


In [23]:
dispersion_df = dispersion_df.assign(
    X=lambda df: df.Frequency,
    Xpos=lambda df: st.Scalers.log_scale(df.X),
    Y=lambda df: df["Rosengren's S"],
    Ypos=lambda df: st.Scalers.scale(df.Y),
)
dispersion_df.head(3)

,Frequency,Range,SD,VC,Juilland's D,Rosengren's S,DP,DP norm,KL-divergence,X,Xpos,Y,Ypos
gates,1,1,0.0274,36.469165,2.220446e-16,0.000754,0.999246,0.999246,10.372865,1,0.0,0.000754,0.0
brass,1,1,0.0274,36.469165,2.220446e-16,0.000754,0.999246,0.999246,10.372865,1,0.0,0.000754,0.0
band,1,1,0.0274,36.469165,2.220446e-16,0.000754,0.999246,0.999246,10.372865,1,0.0,0.000754,0.0


In [28]:
corpus.get_df()

,level_0,index,TokenText,TokenPOS,TokenDep
0,0,0,gates,NOUN,nsubj
1,1,1,brass,NOUN,compound
2,2,2,band,NOUN,pobj
3,3,3,seemed,VERB,ROOT
4,4,4,burning,VERB,amod
...,...,...,...,...,...
1326,1326,1326,Law,NOUN,nsubj
1327,1327,1327,policemen,NOUN,conj
1328,1328,1328,began,VERB,ROOT
1329,1329,1329,shake,VERB,xcomp


In [29]:
html = st.dataframe_scattertext(
    corpus,
    plot_df=dispersion_df,
    metadata=corpus.get_df()['TokenPOS'] + ' (' + corpus.get_df()['TokenDep'].str.upper() + ')',
    ignore_categories=True,
    x_label='Log Frequency',
    y_label="Rosengren's S",
    y_axis_labels=['More Dispersion', 'Medium', 'Less Dispersion'],
)

c:\Source\Repos\scattertext\scattertext\Scalers.py:212: RuntimeWarning: divide by zero encountered in divide
  vec_ss = (vec_ss - vec_ss.min()) * 1. / (vec_ss.max() - vec_ss.min())
c:\Source\Repos\scattertext\scattertext\Scalers.py:212: RuntimeWarning: invalid value encountered in divide
  vec_ss = (vec_ss - vec_ss.min()) * 1. / (vec_ss.max() - vec_ss.min())


In [30]:
open('./dispersion.html', 'wb').write(html.encode('utf-8'))

903986